In [1]:
import sys
import dotenv
dotenv.load_dotenv('../../.env')

sys.path.insert(0, '../')
sys.path.insert(0, '../app')

from app.processing.audio.SIWhisperModel import SIWhisperModel
from app.processing.VideoProcessor import VideoProcessor

whisper = SIWhisperModel('medium', 'whisper-medium')



/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


RuntimeError: CUDA failed with error out of memory

## process youtube

In [3]:
from app.S3Storage import S3Storage


s3 = S3Storage()
yt = VideoProcessor(s3=s3, whisper=whisper, youtube_url="https://www.youtube.com/watch?v=abQFa-Nu5V4")

In [4]:
yt.download_youtube_video("https://www.youtube.com/watch?v=abQFa-Nu5V4")

YT <pytubefix.__main__.YouTube object: videoId=abQFa-Nu5V4>


('/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/documents/youtube/39f061c2-9f1c-4e67-b763-cab925153881.mp4',
 "L'incroyable pouvoir de guérison de l'axolotl")

## process s3 object

In [1]:
from app.S3Storage import S3Storage


s3 = S3Storage()
video = VideoProcessor(s3=s3, whisper=whisper, s3_object_name="youtube/5df9290c-4ba5-4643-a3c2-17ab040b35d3.mp4")
document, chunks = video.extract_document()

ModuleNotFoundError: No module named 'app'

In [ ]:
chunks[0].metadata.word_segments

# tests

In [15]:
import json
import os
from pytubefix import YouTube

po_token_path = "./potoken.json"

def regenerate_po_token(path):
    try:
        import requests
        response = requests.get(os.environ.get("YOUTUBE_TOKEN_GENERATOR_URL"))
        data = response.json()
        print("PO TOKENNN", data)
        with open(path, "w") as f:
            f.write(json.dumps({"access_token": None, "refresh_token": None, "expires": None, "visitorData": data.get('visitor_data'), "po_token": data.get('potoken')}))
    except requests.RequestException as e:
        print(f"Error fetching token: {str(e)}")

regenerate_po_token(po_token_path)

url = "https://www.youtube.com/watch?v=YI05CIh1BTc" # unlisted 
# url = "https://www.youtube.com/watch?v=tBQrFG3OhDU" # public
# url = "https://www.youtube.com/watch?v=oNQ5_R2vNy8" # private
yt = YouTube(url, use_po_token=True, token_file=po_token_path, allow_oauth_cache=True)

playability_status = yt.vid_info.get("playabilityStatus", {})
print(playability_status)

Error fetching token: HTTPConnectionPool(host='youtube-token-generator', port=8007): Max retries exceeded with url: /token (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7ac683f967f0>: Failed to resolve 'youtube-token-generator' ([Errno -3] Temporary failure in name resolution)"))
You can use the tool: https://github.com/YunzheZJU/youtube-po-token-generator, to get the token
{'status': 'OK', 'playableInEmbed': True, 'miniplayer': {'miniplayerRenderer': {'playbackMode': 'PLAYBACK_MODE_ALLOW'}}, 'contextParams': 'Q0FFU0FnZ0M='}


In [ ]:
yt.streams

In [ ]:
from googleapiclient.discovery import build

# Set up YouTube API client
api_key = ''
youtube = build('youtube', 'v3', developerKey=api_key)

def get_channel_videos(channel_id):
    playlist_id = f"{channel_id[0]}U{channel_id[2:]}"
    videos = []
    next_page_token = None
    
    while True:
        res = youtube.playlistItems().list(
            part='snippet',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()
        
        for item in res["items"]:
            videos.append({
                "title": item["snippet"]["title"],
                "video_id": item["snippet"]["resourceId"]["videoId"],
                "publish_time": item["snippet"]["publishedAt"]
            })
        
        next_page_token = res.get('nextPageToken')
        
        if next_page_token is None:
            break
            
    return videos

videos = get_channel_videos("UC1udnO-W6gpR9qzleJ5SDKw")
print(len(videos))

In [1]:
from pytubefix import YouTube
yt = YouTube("https://www.youtube.com/watch?v=YWcoQ8M3cnk",use_po_token=True, token_file="./potoken.json", allow_oauth_cache=True)
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(output_path="./temp_videos", filename="test.mp4")


You can use the tool: https://github.com/YunzheZJU/youtube-po-token-generator, to get the token


'/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/temp_videos/test.mp4'

In [ ]:
video_name = yt.vid_info.get("videoDetails", {}).get("title", "Untitled Video")
video_name

In [ ]:
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(output_path='./', filename='blobremove.mp4')
